![](https://storage.googleapis.com/kaggle-public-downloads/arXiv.JPG)
<h1><center>ArXiv Title Prediction from Abstract</center></h1>

## Introduction

#### In this notebook, we use [BART](https://arxiv.org/abs/1910.13461) Transformer model to perform title generation from abstracts. BART is a sequence-to-sequence model where both the input and targets are text sequences. BART is commonly used for text summarization. In our case, we would want to summarize titles from abstracts.

### Acknowledgements

#### This notebook borrows code from:
* [simpletransformers documentation](https://github.com/ThilinaRajapakse/simpletransformers)
* Andrew Lukyanenko's [Arxiv metadata exploration](https://www.kaggle.com/artgor/arxiv-metadata-exploration) kernel

### Libraries 📚⬇

In [1]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [2]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.8.0.dev20210207+cu110', '4.2.2', '0.9.4')

### Read JSON Data 📝

In [3]:
data_file = '../arxiv_download/tools/arxiv-metadata-oai-snapshot.json'

""" Using `yield` to load the JSON file in a loop to prevent Python memory issues if JSON is loaded directly"""

def get_metadata():
    with open(data_file, 'r') as f:
        for line in f:
            yield line

#### Let's print a sample paper metadata

In [4]:
metadata = get_metadata()
for paper in metadata:
    for k, v in json.loads(paper).items():
        print(f'{k}: {v} \n')
    break

id: 0704.0001 

submitter: Pavel Nadolsky 

authors: C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan 

title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies 

comments: 37 pages, 15 figures; published version 

journal-ref: Phys.Rev.D76:013009,2007 

doi: 10.1103/PhysRevD.76.013009 

report-no: ANL-HEP-PR-07-12 

categories: hep-ph 

license: None 

abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
m

In [5]:
category_map = {'astro-ph': 'Astrophysics',
                'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
                'astro-ph.EP': 'Earth and Planetary Astrophysics',
                'astro-ph.GA': 'Astrophysics of Galaxies',
                'astro-ph.HE': 'High Energy Astrophysical Phenomena',
                'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
                'astro-ph.SR': 'Solar and Stellar Astrophysics',
                'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
                'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
                'cond-mat.mtrl-sci': 'Materials Science',
                'cond-mat.other': 'Other Condensed Matter',
                'cond-mat.quant-gas': 'Quantum Gases',
                'cond-mat.soft': 'Soft Condensed Matter',
                'cond-mat.stat-mech': 'Statistical Mechanics',
                'cond-mat.str-el': 'Strongly Correlated Electrons',
                'cond-mat.supr-con': 'Superconductivity',
                'cs.AI': 'Artificial Intelligence',
                'cs.AR': 'Hardware Architecture',
                'cs.CC': 'Computational Complexity',
                'cs.CE': 'Computational Engineering, Finance, and Science',
                'cs.CG': 'Computational Geometry',
                'cs.CL': 'Computation and Language',
                'cs.CR': 'Cryptography and Security',
                'cs.CV': 'Computer Vision and Pattern Recognition',
                'cs.CY': 'Computers and Society',
                'cs.DB': 'Databases',
                'cs.DC': 'Distributed, Parallel, and Cluster Computing',
                'cs.DL': 'Digital Libraries',
                'cs.DM': 'Discrete Mathematics',
                'cs.DS': 'Data Structures and Algorithms',
                'cs.ET': 'Emerging Technologies',
                'cs.FL': 'Formal Languages and Automata Theory',
                'cs.GL': 'General Literature',
                'cs.GR': 'Graphics',
                'cs.GT': 'Computer Science and Game Theory',
                'cs.HC': 'Human-Computer Interaction',
                'cs.IR': 'Information Retrieval',
                'cs.IT': 'Information Theory',
                'cs.LG': 'Machine Learning',
                'cs.LO': 'Logic in Computer Science',
                'cs.MA': 'Multiagent Systems',
                'cs.MM': 'Multimedia',
                'cs.MS': 'Mathematical Software',
                'cs.NA': 'Numerical Analysis',
                'cs.NE': 'Neural and Evolutionary Computing',
                'cs.NI': 'Networking and Internet Architecture',
                'cs.OH': 'Other Computer Science',
                'cs.OS': 'Operating Systems',
                'cs.PF': 'Performance',
                'cs.PL': 'Programming Languages',
                'cs.RO': 'Robotics',
                'cs.SC': 'Symbolic Computation',
                'cs.SD': 'Sound',
                'cs.SE': 'Software Engineering',
                'cs.SI': 'Social and Information Networks',
                'cs.SY': 'Systems and Control',
                'econ.EM': 'Econometrics',
                'eess.AS': 'Audio and Speech Processing',
                'eess.IV': 'Image and Video Processing',
                'eess.SP': 'Signal Processing',
                'gr-qc': 'General Relativity and Quantum Cosmology',
                'hep-ex': 'High Energy Physics - Experiment',
                'hep-lat': 'High Energy Physics - Lattice',
                'hep-ph': 'High Energy Physics - Phenomenology',
                'hep-th': 'High Energy Physics - Theory',
                'math.AC': 'Commutative Algebra',
                'math.AG': 'Algebraic Geometry',
                'math.AP': 'Analysis of PDEs',
                'math.AT': 'Algebraic Topology',
                'math.CA': 'Classical Analysis and ODEs',
                'math.CO': 'Combinatorics',
                'math.CT': 'Category Theory',
                'math.CV': 'Complex Variables',
                'math.DG': 'Differential Geometry',
                'math.DS': 'Dynamical Systems',
                'math.FA': 'Functional Analysis',
                'math.GM': 'General Mathematics',
                'math.GN': 'General Topology',
                'math.GR': 'Group Theory',
                'math.GT': 'Geometric Topology',
                'math.HO': 'History and Overview',
                'math.IT': 'Information Theory',
                'math.KT': 'K-Theory and Homology',
                'math.LO': 'Logic',
                'math.MG': 'Metric Geometry',
                'math.MP': 'Mathematical Physics',
                'math.NA': 'Numerical Analysis',
                'math.NT': 'Number Theory',
                'math.OA': 'Operator Algebras',
                'math.OC': 'Optimization and Control',
                'math.PR': 'Probability',
                'math.QA': 'Quantum Algebra',
                'math.RA': 'Rings and Algebras',
                'math.RT': 'Representation Theory',
                'math.SG': 'Symplectic Geometry',
                'math.SP': 'Spectral Theory',
                'math.ST': 'Statistics Theory',
                'math-ph': 'Mathematical Physics',
                'nlin.AO': 'Adaptation and Self-Organizing Systems',
                'nlin.CD': 'Chaotic Dynamics',
                'nlin.CG': 'Cellular Automata and Lattice Gases',
                'nlin.PS': 'Pattern Formation and Solitons',
                'nlin.SI': 'Exactly Solvable and Integrable Systems',
                'nucl-ex': 'Nuclear Experiment',
                'nucl-th': 'Nuclear Theory',
                'physics.acc-ph': 'Accelerator Physics',
                'physics.ao-ph': 'Atmospheric and Oceanic Physics',
                'physics.app-ph': 'Applied Physics',
                'physics.atm-clus': 'Atomic and Molecular Clusters',
                'physics.atom-ph': 'Atomic Physics',
                'physics.bio-ph': 'Biological Physics',
                'physics.chem-ph': 'Chemical Physics',
                'physics.class-ph': 'Classical Physics',
                'physics.comp-ph': 'Computational Physics',
                'physics.data-an': 'Data Analysis, Statistics and Probability',
                'physics.ed-ph': 'Physics Education',
                'physics.flu-dyn': 'Fluid Dynamics',
                'physics.gen-ph': 'General Physics',
                'physics.geo-ph': 'Geophysics',
                'physics.hist-ph': 'History and Philosophy of Physics',
                'physics.ins-det': 'Instrumentation and Detectors',
                'physics.med-ph': 'Medical Physics',
                'physics.optics': 'Optics',
                'physics.plasm-ph': 'Plasma Physics',
                'physics.pop-ph': 'Popular Physics',
                'physics.soc-ph': 'Physics and Society',
                'physics.space-ph': 'Space Physics',
                'q-bio.BM': 'Biomolecules',
                'q-bio.CB': 'Cell Behavior',
                'q-bio.GN': 'Genomics',
                'q-bio.MN': 'Molecular Networks',
                'q-bio.NC': 'Neurons and Cognition',
                'q-bio.OT': 'Other Quantitative Biology',
                'q-bio.PE': 'Populations and Evolution',
                'q-bio.QM': 'Quantitative Methods',
                'q-bio.SC': 'Subcellular Processes',
                'q-bio.TO': 'Tissues and Organs',
                'q-fin.CP': 'Computational Finance',
                'q-fin.EC': 'Economics',
                'q-fin.GN': 'General Finance',
                'q-fin.MF': 'Mathematical Finance',
                'q-fin.PM': 'Portfolio Management',
                'q-fin.PR': 'Pricing of Securities',
                'q-fin.RM': 'Risk Management',
                'q-fin.ST': 'Statistical Finance',
                'q-fin.TR': 'Trading and Market Microstructure',
                'quant-ph': 'Quantum Physics',
                'stat.AP': 'Applications',
                'stat.CO': 'Computation',
                'stat.ME': 'Methodology',
                'stat.ML': 'Machine Learning',
                'stat.OT': 'Other Statistics',
                'stat.TH': 'Statistics Theory'}

# #1. Abstract Summarization
###    Title Prediction from Abstract using **BART**

#### For this task, let's choose all paper categories published in the last decade

In [6]:
titles = []
abstracts = []

# Consider all categories in the `category_map` to be used during training and prediction
paper_categories = np.array(list(category_map.keys())).flatten()

# # Consider specific paper categories to be used during training and prediction
# paper_categories = ["cs.AI", # Artificial Intelligence
#                     "cs.CV", # Computer Vision and Pattern Recognition
#                     "cs.LG"] # Machine Learning

metadata = get_metadata()
for paper in tqdm(metadata):
    paper_dict = json.loads(paper)
    category = paper_dict.get('categories')
    try:
        year = int(paper_dict.get('journal-ref')[-4:])
        if category in paper_categories and 2010<year<2021:
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract').replace("\n",""))
    except:
        pass 

len(titles), len(abstracts)

(19973, 19973)

### #1.2 Data Pre-processing ⚙️

In [7]:
papers = pd.DataFrame({
    'title': titles,
    'abstract': abstracts,
})

papers = papers[['abstract', 'title']]
papers.columns = ['input_text', 'target_text']
papers = papers.dropna()

del titles, abstracts
papers

,input_text,target_text
0,We present in this article a detailed quanti...,A unified analysis of the reactor neutrino pro...
1,The aim of the KArlsruhe TRItium Neutrino ex...,The KATRIN sensitivity to the neutrino mass an...
2,"In this letter, we propose three different s...","Penguin-mediated B_(d,s)->VV decays and the Bs..."
3,We find non-supersymmetric metastable vacua ...,Non Supersymmetric Metastable Vacua in N=2 SYM...
4,We show how to test whether a graph with n v...,Recognizing Partial Cubes in Quadratic Time
...,...,...
19968,"Schroedinger (Nature, v.169, p.538 (1952)) d...",Real-Valued Charged Fields and Interpretation ...
19969,"Given a collection of states (rho_1, ..., rh...",How many copies are needed for state discrimin...
19970,"Hu, Paz and Zhang [ B.L. Hu, J.P. Paz and Y....",Alternative Derivation of the Hu-Paz-Zhang Mas...
19971,We demonstrate guiding of cold neutral atoms...,Guiding Neutral Atoms with a Wire


### #1.3 Model Training & Evaluation 🚞

In [8]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

eval_df = papers.sample(frac=0.1, random_state=42)
train_df = papers.drop(eval_df.index)

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 6,
    "num_train_epochs": 3,
}

# Create a Bart-base model
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

CPU times: user 5.25 s, sys: 349 ms, total: 5.6 s
Wall time: 6.04 s


In [9]:
%%time

# Train the model
model.train_model(train_df)

# Evaluate the model
result = model.eval_model(eval_df)
print(result)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model: Training started



INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-2000



INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-4000



INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-6000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-8000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/


INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 2.1650455231666563}

{'eval_loss': 2.1650455231666563}
CPU times: user 40min 7s, sys: 5.54 s, total: 40min 13s
Wall time: 25min 54s


### #1.4 Prediction 🔮

In [10]:
document = []

for _ in range(250):
    data = dict()

    random_idx = random.randint(0, len(eval_df)-1)

    abstract = eval_df.iloc[random_idx]['input_text']
    true_title = eval_df.iloc[random_idx]['target_text']

    # Predict with trained BART model
    predicted_title = model.predict([abstract])[0]

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title: {predicted_title}\n')
    print(f'Abstract: {abstract}\n\n\n')

    data['true_title'] = true_title
    data['pred_title'] = predicted_title
    data['abstract'] = abstract

    document.append(data)

import json

with open('result.json', 'w') as f:
    json.dump(document, f)




True Title: Electroweak Theory with a Minimal Length

Predicted Title: Gauge theory with gauge group SU(N)

Abstract:   According to the introduction of a minimal length to quantum field theorywhich is directly related to a generalized uncertainty principle theimplementation of the gauge principle becomes much more intricated. It has beenshown in another paper how gauge theories have to be extended in general, ifthere is assumed the existence of a minimal length. In this paper thisgeneralization of the description of gauge theories is applied to the case ofYang-Mills theories with gauge group SU(N) to consider especially theapplication to the electroweak theory as it appears in the standard model. Themodifications of the lepton-, Higgs- and gauge field sector of the extendedLagrangian of the electroweak theory maintaining local gauge invariance under$SU(2)_L\otimes U(1)_Y$ transformations are investigated. There appearadditional interaction terms between the leptons or the Higgs parti


True Title: Local Factors, Reciprocity and Vinberg Monoids

Predicted Title: Local factors and monoids

Abstract:   This article addresses the problem of existence of local factors, i.e., theroot numbers and L-functions attached to representations of reductive groupsover local fields and irreducible finite dimensional representations of theirL-groups, as well as their equality with those of Artin factors through thelocal Langlands correspondence. We conclude the paper with a survey of thetheory of monoids of Braverman-Kazhdan, Ngo and Vinberg in generalizing themethod of Godement and Jacquet to arbitrary setting and their connections withLanglands-Shahidi method.






True Title: Measurement of CP violation in Bs --> J/psi phi decay

Predicted Title: Measurement of CP violation in Bs --> J/psi phi decay

Abstract:   We briefly discuss measurements of CP violation in Bs --> J/psi phi decay.Both the phenomenology of Bs mixing and the importance of the measurement tosearches for new physics, as well as technical details and issues with theanalysis are included. While current results are consistent with the standardmodel, even large contributions from new physics cannot be excluded.






True Title: PAAI in the sky : towards a particulate mechanism for Dark Energy and
  concordant Dark Matter

Predicted Title: Dark Energy in a Hidden Sector with Very Light fermions

Abstract:   We propose the origins of Dark Energy in a hidden sector with a pair of verylight fermions, oppositely charged under an abelian gauge force $U(1)_X$ but ofunequal mass. The system is dubbed PAAI, plasma which is abelian, asymmetricand idealised. For a range of the hidden fine structure constant values and thevalue of mass of the lightest fermion the PAAI is argued to simulate DarkEnergy. Additional fermions from the same sector are shown to account for DarkMatter. Further, residual $X$-magnetic fields can mix with Maxwellelectromagnetism to provides the seed for cosmic-scale magnetic fields. Thusthe scenario can explain several cosmological puzzles from within the samehidden sector.






True Title: Cooling in a parametrically driven optomechanical cavity

Predicted Title: Master Equation for a Parametrically Driven Mechanical Cavity

Abstract:   We obtain a master equation for a parametrically driven optomechanicalcavity. We use a more correct dissipation model that accounts for themodification of the quasienergy spectrum caused by the driving. When thenatural frequency of the mechanical object oscillates periodically around itsmean value, the master equation with the improved dissipation model isexpressed using Floquet operators. We apply the corresponding master equationto model the laser cooling of the mechanical object. Using an adiabaticapproximation, an analytical expression for the number of excitations of themechanical oscillator can be obtained. We find that the number of excitationscan be lower than in the non-time-dependent case. Our results raise thepossibility of achieving lower temperatures for the mechanical object if itsnatural frequency can be contro

...


True Title: Y-Net: A deep Convolutional Neural Network for Polyp Detection

Predicted Title: A Novel Deep Learning Approach for Colorectal Polyps Detection in
  Colonoscopy

Abstract:   Colorectal polyps are important precursors to colon cancer, the third mostcommon cause of cancer mortality for both men and women. It is a disease whereearly detection is of crucial importance. Colonoscopy is commonly used forearly detection of cancer and precancerous pathology. It is a demandingprocedure requiring significant amount of time from specialized physicians andnurses, in addition to a significant miss-rates of polyps by specialists.Automated polyp detection in colonoscopy videos has been demonstrated to be apromising way to handle this problem. {However, polyps detection is achallenging problem due to the availability of limited amount of training dataand large appearance variations of polyps. To handle this problem, we propose anovel deep learning method Y-Net that consists of two encoder 


True Title: Small influence of magnetic ordering on lattice dynamics in
  TaFe$_{1.25}$Te$_3$

Predicted Title: Raman scattering spectra of zigzag spin chain TaFe$_{1.25}

Abstract:   Raman scattering spectra of zigzag spin chain TaFe$_{1.25}$Te$_3$ singlecrystal are presented in a temperature range from 80 to 300 K. Nine Ramanactive modes of $A_g$ and $B_g$ symmetry are clearly observed and assigned byprobing different scattering channels, which is confirmed by lattice dynamicscalculations. Temperature dependence of the Raman modes linewidth is mainlygoverned by the lattice anharmonicity. The only deviation from the conventionalbehavior is observed for $A_g$ symmetry modes in a vicinity of the magneticphase transition at $T_N \approx 200$ K. This implies that the electron-phononinteraction weakly changes with temperature and magnetic ordering, whereassmall changes in the spectra near the critical temperature can be ascribed tospin fluctuations.






True Title: Enhanced and tunable electric dipole-dipole interaction near to planar
  metal film

Predicted Title: Enhanced electric dipole-dipole interaction mediated by surface plasmon polaritons


Abstract:   We investigate the enhanced electric dipole-dipole interaction by surfaceplasmon polaritons (SPPs) supported by the planar metal film waveguide. Bytaking two nitrogen-vacancy (NV) center electric-dipoles in diamond as anexample, both the coupling strength and collective relaxation of two dipolesare studied with numerical Green Function method. Compared to the two-dipolecoupling on planar surface, metal film provides stronger and tunable couplingcoefficients. Enhancement of the interaction between coupled NV center dipolescould have applications in both quantum information and energy transferinvestigation. Our investigation provides a systematical result forexperimental applications based on dipole-dipole interaction mediated with SPPson planar metal film.






True Title: Multi-View Bayesian Correlated Component Analysis

Predicted Title: Correlated Component Analysis in Multi-View Brain

Abstract:   Correlated component analysis as proposed by Dmochowski et al. (2012) is atool for investigating brain process similarity in the responses to multipleviews of a given stimulus. Correlated components are identified under theassumption that the involved spatial networks are identical. Here we propose ahierarchical probabilistic model that can infer the level of universality insuch multi-view data, from completely unrelated representations, correspondingto canonical correlation analysis, to identical representations as incorrelated component analysis. This new model, which we denote Bayesiancorrelated component analysis, evaluates favourably against three relevantalgorithms in simulated data. A well-established benchmark EEG dataset is usedto further validate the new model and infer the variability of spatialrepresentations across multiple subject


True Title: Kadison's Pythagorean Theorem and essential codimension

Predicted Title: On the integrality condition of diagonals of projections

Abstract:   Kadison's Pythagorean theorem (2002) provides a characterization of thediagonals of projections with a subtle integrality condition. Arveson (2007),Kaftal, Ng, Zhang (2009), and Argerami (2015) all provide different proofs ofthat integrality condition. In this paper we interpret the integralitycondition in terms of the essential codimension of a pair of projectionsintroduced by Brown, Douglas and Fillmore (1973), or, equivalently of the indexof a Fredholm pair of projections introduced by Avron, Seiler, and Simon(1994). The same techniques explain the integer occurring in thecharacterization of diagonals of selfadjoint operators with finite spectrum byBownik and Jasper (2015).



